# Imports

In [ ]:

from sklearn import preprocessing

from models.neural_net import NeuralNet
from view.visualizer import *


# Dataset creation

In [ ]:
xd = 3  # Input space dimensions
yd = 4  # Output dimension
num_samples = 50  # Number of data points
epochs = 100  # Number of training epochs
l = [10]  # Number of neurons in each layer

dataset = Dataset()
dataset.create_dataset_sinus_2d(num_samples)
dataset.scale(preprocessing.MinMaxScaler(feature_range=(0, 2)))
dataset.split_train_test(0.2)


# Neural Network training with Adam

In [ ]:
model_nn = NeuralNet()
model_nn.fit(dataset.X_train, dataset.y_train, l, validation_split=0.2, epochs=1)
y_nn_train = model_nn.predict(dataset.X_train)
y_nn_test = model_nn.predict(dataset.X_test)


# Sampled Netwrok training.
### very first algorithm


In [ ]:
model_sampled = SampledNet()
x_1_X2_tuples = model_sampled.fit(dataset.X_train, y_nn_train, model_nn, layer2="classic", choose_x_2="angle")

model_nn_vs_model_sampled(dataset, model_nn, model_sampled, x_1_X2_tuples)

### add radius: an interval for the distance to the bias origin, and choose x_2, so that \hat{w_i} has the closest norm to w_i

In [ ]:
model_sampled = SampledNet()
x_1_X2_tuples = model_sampled.fit(dataset.X_train, y_nn_train, model_nn, layer2="classic", radius=0,
                                  choose_x_2="norm")
model_nn_vs_model_sampled(dataset, model_nn, model_sampled, x_1_X2_tuples)


### shift x_1 and x_2 so that x_1 lies on the original bias origin

In [ ]:
model_sampled = SampledNet()
x_1_X2_tuples = model_sampled.fit(dataset.X_train, y_nn_train, model_nn, layer2="classic", radius=0.09,
                                  choose_x_2="norm", project_onto_boundary=True)

model_nn_vs_model_sampled(dataset, model_nn, model_sampled, x_1_X2_tuples)


### use ridge regression for layer 2

In [ ]:
model_sampled = SampledNet()
model_sampled.fit(dataset.X_train, y_nn_train, model_nn, layer2="ridge", radius=0.05,
                  choose_x_2="norm")

model_nn_vs_model_sampled(dataset, model_nn, model_sampled, x_1_X2_tuples)


### Calculate radius vs loss.

In [ ]:

y_nn_train = model_nn.predict(dataset.X_train)
y_nn_test = model_nn.predict(dataset.X_test)
weight1 = []
weight2 = []
losses = []
radii = np.linspace(0, 0.5, 10)
for radius in radii:
    model_sampled = SampledNet()
    model_sampled.fit(
        dataset.X_train, y_nn_train, model_nn, layer2="classic", radius=radius, alpha=-1, verbose=0
    )
    weight1.append(model_sampled.weights[0])
    weight2.append(model_nn.weights[0])
    y_sampled_test = model_sampled.predict(dataset.X_test)
    loss_sampled_nn = loss_mse(y_sampled_test, y_nn_test)
    losses.append(loss_sampled_nn)
    print(loss_sampled_nn)

In [ ]:
plot_vector_differences(weight1, weight2, radii, losses)

### augment data with gaussian sampling

In [ ]:
xd = 3  # Input space dimensions
yd = 4  # Output dimension
num_samples = 50  # Number of data points
epochs = 100  # Number of training epochs
l = [100]  # Number of neurons in each layer

dataset = Dataset()
dataset.create_dataset_sinus_2d(num_samples)
dataset.scale(preprocessing.MinMaxScaler(feature_range=(-2, 2)))
dataset.split_train_test(0.2)

model_nn = NeuralNet()
model_nn.fit(dataset.X_train, dataset.y_train, l, validation_split=0.2, epochs=1)
y_nn_train = model_nn.predict(dataset.X_train)
y_nn_test = model_nn.predict(dataset.X_test)

model_sampled = SampledNet()
x_1_X2_tuples = model_sampled.fit(dataset.X_train, y_nn_train, model_nn, layer2="classic", radius=0.05,
                                  choose_x_2="norm")

model_sampled_input_augmented = SampledNet()
x_1_X2_tuples_input_augmented = model_sampled_input_augmented.fit(dataset.X_train, y_nn_train, model_nn,
                                                                  layer2="classic",
                                                                  radius=0.05,
                                                                  choose_x_2="norm", augment_data=(1, 10))

model_nn_vs_model_sampled(dataset, model_nn, model_sampled, x_1_X2_tuples)
model_nn_vs_model_sampled(dataset, model_nn, model_sampled_input_augmented, x_1_X2_tuples_input_augmented)
